# Fake Image Detector

## Imports

In [ ]:
%matplotlib inline 
import urllib, cStringIO
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pandas as pd
import urllib2
import scipy
import scipy.misc
from scipy import spatial as sp
from os import listdir
from os.path import isfile, join

## Tinkering

In [ ]:
aze = "N61063" #"1A2X6W"
URL = 'http://fr.louisvuitton.com/images/is/image/lv/1/PP_VP_L/louis-vuitton--'+aze+'_PM2_Front%20view.jpg'
file = cStringIO.StringIO(urllib.urlopen(URL).read())
img = Image.open(file)
img.thumbnail((256, 256), Image.ANTIALIAS) # resizes image in-place
imgplot = plt.imshow(img)